In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()">
<input type="submit" value="Click here to view the raw code"></form>''')

# Ticker Plotter
Start by selecting the Holding of interest and defining a period time along the interval

In [2]:
import os
import sys
from inspect import getsourcefile

CURRENT_PATH = os.path.abspath(getsourcefile(lambda: 0))
ROOT_DIR = os.path.dirname(os.path.dirname(CURRENT_PATH))
PARENT_DIR = ROOT_DIR[: ROOT_DIR.rfind(os.path.sep)]
sys.path.insert(0, os.path.join(PARENT_DIR, "sources_repo/yfinance"))

In [3]:
import api_requests as ar
import chart_studio.plotly as py
import plotly.graph_objs as go
import cufflinks as cf
import pandas as pd
import yfinance as yf
import holding_util as hu
import numpy as np
import plotly.express as px

from ipywidgets import widgets, interact, interact_manual, HTML
from IPython.display import clear_output, display
from traitlets import traitlets

#Information
path=r"C:\Users\Marco\Documents"
file_name="investment_shares.csv"
df = ar.load_csv(path, file_name)
holdings_list = ar.create_shares_list(df)

plotting_time = {"period":{"1 day": "1d",
                           "5 days": "5d",
                           "1 month": "1mo",
                           "3 months": "3mo",
                           "6 months": "6mo",
                           "1 year": "1y",
                           "2 years": "2y",
                           "5 years": "5y",
                           "10 years": "10y",
                           "Year-to-date": "ytd",
                           "Max": "max"},
                "interval": {"1 minute": "1m",
                             "5 minutes": "5m",
                             "30 minutes": "30m",
                             "60 minutes": "60m",
                             "1 hour": "1h",
                             "1 day": "1d",
                             "5 days": "5d",
                             "1 week": "1wk",
                             "1 month": "1mo"}}

per_value_list = list(plotting_time.get('period').values())
per_key_list = list(plotting_time.get('period'))
period_list = list(zip(per_key_list, per_value_list))
inter_value_list = list(plotting_time.get('interval').values())
inter_key_list = list(plotting_time.get('interval'))
interval_list = list(zip(inter_key_list, inter_value_list))

In [4]:
#Dropdowns
ticker_dd = widgets.Dropdown(options=holdings_list,
                           description="Holding:",
                           disabled=False,
                           value=None)

period_dd = widgets.Dropdown(options=period_list,
                             description="Period:",
                             disabled=False,
                             value=None)

interval_dd = widgets.Dropdown(options=interval_list,
                                description="Interval:",
                                disabled=False,
                                value=None)

def on_plotting_change(change):
    if period_dd.value and interval_dd.value:
        clear_output()
        display(ticker_dd, period_dd, interval_dd)
        on_ticker_change()
        try:
            tick = yf.Ticker(str(ticker))
            ticker_plot(tick, period=period_dd.value, interval=interval_dd.value)
        except:
            print("Please select a Holding value")
        
def on_global_variable_change(change):
    global ticker
    ticker = ticker_dd.value
    
def on_ticker_change():
    if ticker_dd.value:
        clear_output()
        display(ticker_dd, period_dd, interval_dd)
        try:
            tick = yf.Ticker(ticker_dd.value)
            info = tick.info
            info_dict = {}
            value_list = ['shortName', 'sector']
            info_dict = {"Name": info.get('shortName'),
                         "Sector": info.get('sector'),
                         "Industry": info.get('industry'),
                         "Market Price": "{} {}".format(info.get('currency'),
                                                       info.get('regularMarketPrice')),
                         "P/E": info.get('pegRatio'),
                         "Payout Ratio": info.get('payoutRatio'),
                         "Div Yield": info.get('dividendYield'),
                         "Market": info.get('exchange')}
            info_df = pd.DataFrame.from_dict(info_dict,
                                             orient='index',
                                             columns=[""])
            display(info_df)
        except (IndexError, AttributeError):
            print("No information available for {}".format(str(ticker_dd.value)))

def ticker_plot(tick, period="1y", interval="1d"):
    hist = tick.history(period=period, interval=interval)
    fig = go.Figure()
    fig.add_trace(go.Scatter(y=hist.Close,
                             x=hist.index,
                             name=str(ticker.upper())))
    fig.update_layout(title={'text': str(ticker.upper()),
                             'y':0.9,
                             'x':0.5,
                             'xanchor': 'center',
                             'yanchor': 'top'},
                      yaxis_title="Price [{}]".format(tick.info.get('currency')))
    display(py.iplot(fig))
    
ticker_dd.observe(on_global_variable_change, names="value")
period_dd.observe(on_plotting_change, names="value")
interval_dd.observe(on_plotting_change, names="value")
display(ticker_dd, period_dd, interval_dd)


Dropdown(description='Holding:', options=('ATVI', 'WM', 'NOBL', 'VOO', 'MU', 'BAM', 'QQQ', 'YUM', 'HPQ', 'LAZ'…

Dropdown(description='Period:', options=(('1 day', '1d'), ('5 days', '5d'), ('1 month', '1mo'), ('3 months', '…

Dropdown(description='Interval:', options=(('1 minute', '1m'), ('5 minutes', '5m'), ('30 minutes', '30m'), ('6…

# ETF Portfolio
Select an ETF to see its top holdings

In [16]:
#ETF Portfolio
etf_df = ar.df_by_type(df, "ETF")
etf_list = etf_df.holding

etf_dd = widgets.Dropdown(options=etf_list,
                          description="ETF:",
                          disabled=False,
                          value=None)

def on_etf_change(change):
    if etf_dd.value:
        global holdings_df
        global tickers_list
        clear_output()
        display(etf_dd)
        print("Retrieving data for {}...".format(etf_dd.value))
        tick = hu.Holding(etf_dd.value)
        holdings_df = tick.scrape_holdings(source="yahoo finance")
        hold_tot_perc = round(sum(holdings_df["% Assets"]), 2)

#         #Using Tickers
#         tickers_list = " ".join(holdings_df.index)
#         tickers = hu.Holdings(tickers_list)
#         holdings_df['Sector'] = tickers.return_sector_list()
        
        #Using ScrapeHolding
        tickers_list = holdings_df.index
        sh = hu.ScrapeHolding(holdings_list=tickers_list)
        holdings_df["Sector"] = sh.scrape_json_values(detail_key="sector")
        
        sum_percent = round(sum(holdings_df["% Assets"]), 2)
        
        display(holdings_df)
        print("Top {} holdings represent {}% of total assets".format(len(holdings_df.index), hold_tot_perc))
        row = pd.DataFrame([("Other", 100-sum_percent, "Other")],
                           columns=["Name", "% Assets", "Sector"],
                           index=["Other"])
        asset_other_df = holdings_df.append(row, sort=True)
        holdings_pie(asset_other_df)
        
        holdings_df["Country"] = sh.scrape_json_values(detail_key="country")
        countries_map(holdings_df["Country"])
        
        
def holdings_pie(df):
    labels = df["Sector"]
    values = df["% Assets"]
    fig = go.Figure(data=[go.Pie(labels=labels,
                                 values=values,
                                 hoverinfo='label+percent',
                                 textinfo='label+percent')])
    fig.show()
    

def countries_map(country_list):
    country_df = pd.DataFrame()
    country_df["Country"] = country_list
    country_df["Country Code"] = ar.return_alpha3_list(country_list)
    new_df = holdings_df.iloc[:-1].reset_index()
    country_df['% Assets'] = new_df["% Assets"]
    group_df = country_df.groupby('Country Code').sum().reset_index()
    group_df["Country"] = ar.return_country_name_list(group_df["Country Code"])

    fig = px.choropleth(group_df,
                        locations="Country Code",
    #                     locationmode = "country names",
                        color="% Assets",
                        hover_name="Country",
                        color_continuous_scale=px.colors.sequential.Purpor,
                        scope='world')
    # fig = go.Figure(data=go.Choropleth(locations=group_df["Country Code"],
    #                                    z = group_df["% Assets"],
    #                                    colorscale = 'Blues',
    #                                    autocolorscale = True,
    # #                                    text = group_df["Country"],
    #                                    marker_line_color = 'black',
    #                                    colorbar_ticksuffix = "%",
    #                                    marker_line_width = 0.75,))
    fig.update_layout(
        title_text = "Percent of Top {} Assets by Country".format(len(country_list)),
        title_x = 0.5,
        geo=dict(showframe = False,
                 showcoastlines = False,
                 projection_type = 'equirectangular'))
    fig.show()


etf_dd.observe(on_etf_change, names="value")
display(etf_dd)


Dropdown(description='ETF:', index=5, options=('NOBL', 'VOO', 'QQQ', 'BOTZ', 'VXUS', 'VYMI'), value='VYMI')

Retrieving data for VYMI...


,Name,% Assets,Sector
Symbol,,,
2330.TW,Taiwan Semiconductor Manufacturing Co Ltd,2.47,Technology
7203,Toyota Motor Corp,1.72,None
HSBA.L,HSBC Holdings PLC,1.48,Financial Services
AZN.L,AstraZeneca PLC,1.29,Healthcare
BP..L,BP PLC,1.21,None
RDSA.L,Royal Dutch Shell PLC Class A,1.19,Energy
FP.PA,Total SA,1.18,Energy
GSK.L,GlaxoSmithKline PLC,1.16,Healthcare
RY.TO,Royal Bank of Canada,1.15,Financial Services


Top 10 holdings represent 13.9% of total assets


In [6]:
# sh = hu.ScrapeHolding(holdings_list=tickers_list)
# country_list = sh.scrape_json_values(detail_key="country")

In [7]:
# def countries_map(country_list):
#     country_df = pd.DataFrame()
#     country_df["Country"] = country_list
#     country_df["Country Code"] = ar.return_alpha3_list(country_list)
#     new_df = holdings_df.iloc[:-1].reset_index()
#     country_df['% Assets'] = new_df["% Assets"]
#     group_df = country_df.groupby('Country Code').sum().reset_index()
#     group_df["Country"] = ar.return_country_name_list(group_df["Country Code"])

#     fig = px.choropleth(group_df,
#                         locations="Country Code",
#     #                     locationmode = "country names",
#                         color="% Assets",
#                         hover_name="Country",
#                         color_continuous_scale=px.colors.sequential.Purpor,
#                         scope='world')
#     # fig = go.Figure(data=go.Choropleth(locations=group_df["Country Code"],
#     #                                    z = group_df["% Assets"],
#     #                                    colorscale = 'Blues',
#     #                                    autocolorscale = True,
#     # #                                    text = group_df["Country"],
#     #                                    marker_line_color = 'black',
#     #                                    colorbar_ticksuffix = "%",
#     #                                    marker_line_width = 0.75,))
#     fig.update_layout(
#         title_text = "Percent of Top {} Assets by Country".format(len(country_list)),
#         title_x = 0.5,
#         geo=dict(showframe = False,
#                  showcoastlines = False,
#                  projection_type = 'equirectangular'))
#     fig.show()



In [8]:
# country_df = pd.DataFrame()
# country_df["Country Code"] = ar.return_alpha3_list(country_list)
# new_df = holdings_df.iloc[:-1].reset_index()
# country_df['% Assets'] = new_df["% Assets"]
# # group_df = country_df.groupby('Country Code').sum()
# # group_df
# country_df

In [9]:
# #CSV scraping
# url = 'https://www.ishares.com/us/products/239705/ishares-phlx-semiconductor-etf/\
# 1467271812596.ajax?fileType=csv&fileName=SOXX_holdings&dataType=fund'

# df = pd.read_csv(url, skiprows=9)    
# df.head()


In [10]:
# #Creating Stocks dataframe
# df = ar.compile_shares_df(df)
# stock_df = ar.df_by_type(df, 'Stock')
# weekend_check = ar.is_weekend()
# history_df = ar.create_history_summary(stock_df.holding, period='1y', interval='1d')
# last_close_value = ar.last_close(history_df)
# combined_df = ar.join_shares_history_df(stock_df, history_df)
# combined_df = ar.calc_share_gain(combined_df, weekend_check)
# combined_df.head()

In [11]:
# interact_manual=interact_manual.options(manual_name="Run")
# @interact_manual(Holding=df_btn.holdings_list)
# def ticker_info(Holding):
#     tick = yf.Ticker(Holding)
#     info = tick.info
#     info_dict = {"name": info['shortName'],
#                  "sector": info['sector'],
#                  "industry": info['industry'],
#                  "payout_perc": round(info['payoutRatio'] * 100, 2),
#                  "div_yield_perc": round(info['dividendYield'] * 100, 2),
#                  "xchange_mkt": info['exchange'],
#                  "pe_ratio": info['pegRatio'],
#                  "price_USD": info['regularMarketPrice']}
#     info_df = pd.DataFrame.from_dict(info_dict,
#                                      orient='index',
#                                      columns=[str(Holding)])
#     return info_df

In [12]:
# class LoadedButton(widgets.Button):
#     """A button that can holds a value as a attribute."""

#     def __init__(self, path=None, file_name=None, df=None, holding_list=None,*args, **kwargs):
#         super(LoadedButton, self).__init__(*args, **kwargs)
#         # Create the value attribute.
#         self.path=path
#         self.file_name=file_name
#         self.add_traits(df=traitlets.Any(df))
#         self.add_traits(holding_list=traitlets.Any(holding_list))
        
# def return_holding_list(ex):
#     clear_output()
#     display(df_btn)
#     ex.df = ar.load_csv(ex.path, ex.file_name)
#     ex.holding_list = ex.df['holding']

# df_btn = LoadedButton(description="Load CSV",
#                       path=path,
#                       file_name=file_name)
# df_btn.on_click(return_holding_list)
# display(df_btn)

In [13]:
# import yfinance.utils as utils
# scrape_url = 'https://finance.yahoo.com/quote'
# url = '%s/%s' % (scrape_url, 'AMD')
# data = utils.get_json(url)
# data['summaryProfile']['country']